In [1]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

In [2]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")

χ (generic function with 1 method)

In [123]:
domain = (0,1,0,1)
partition = (50,50)
model = CartesianDiscreteModel(domain,partition) |> simplexify
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1) ###### conformity correct?
Trialspace = TransientTrialFESpace(Testspace)                                # maybe add a function for/if Dirichlet conditions



Uspace = FESpace(model, reffe, conformity=:H1)

degree = 2*order                                                    # degree of the method used for approximating integrals over Ω
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)      										# make the measure dΩ
Γ = BoundaryTriangulation(model)                                    # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)   


q_pos(x) = χ(x[1], -0.9, -0.85) * χ(x[2], 0, 0.20)

ρ=1.225
c = 1# c=1020.0
k = 1# k=15.0
h = 0.1# h=0.7
Tout(x,t)=5.0
Q(x,t) = 0 #100.0*q_pos(x)
Tini(x) = 0.0
t0=0.0
tF=5.0
TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))
Δt = 0.05

γ = 1

1

In [124]:
ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

Q(t)=x->Q(x,t)
Tout(t)=x->Tout(x,t)

a_SE_tconst(t, dtT, ϕ) = ∫(c*dtT*ϕ*ρ)dΩ
a_SE_tnonconst(t, T, ϕ) = (-1)*∫(k * ∇(T) ⋅ ∇(ϕ) + tr(h*T*ϕ))dΩ
# a_SE_tnonconst(t, T, ϕ) = (-1)*∫(k * ∇(T)⊙∇(ϕ))dΩ - ∫(h*T*ϕ)dΓ


l_SE(t, ϕ) = ∫(Q(t) * ϕ)dΩ - ∫(Tout(t) * ϕ * h)dΓ
op_SE = TransientLinearFEOperator((a_SE_tconst, a_SE_tnonconst), l_SE, Trialspace, Testspace, constant_forms=(true, false))

T = solve(solver, op_SE, t0, tF, TIni)

sol = []
count = 0
for (tn, uhn) in T
    if count%5 == 0
        push!(sol,uhn)
    end
    count += 1

end


In [125]:
fig, ax, hm = plot(sol[1])
Colorbar(fig[1,2], hm)
display(fig)

CairoMakie.Screen{IMAGE}


In [126]:
solution_animation(sol,30)

ErrorException: Can't interpolate in a range where cmin == cmax. This can happen, for example, if a colorrange is set automatically but there's only one unique value present.